In [0]:
import os
from pathlib import Path
import multiprocessing as proc
import numpy as np
import pandas as pd
from skimage.data import imread
from sklearn.ensemble import RandomForestClassifier
from subprocess import check_output
import pickle

camera_model.pkl
test1
train



In [0]:
train_path = '/home/maansi/input/train'
test_path = '/home/maansi/input/test1'

In [0]:
cameras = os.listdir(train_path)
train_images = []
for camera in cameras:
    for fname in sorted(os.listdir(train_path / camera)):
        train_images.append((camera, fname))
train = pd.DataFrame(train_images, columns=['camera', 'fname'])

(2750, 2)


,camera,fname
68,HTC-1-M7,(HTC-1-M7)160.jpg
1155,iPhone-6,(iP6)149.jpg
1622,iPhone-4s,(iP4s)74.jpg
447,Motorola-Nexus-6,(MotoNex6)254.jpg
358,Motorola-Nexus-6,(MotoNex6)174.jpg


In [0]:
test_images = []
for fname in sorted(os.listdir(test_path)):
    test_images.append(fname)
test = pd.DataFrame(test_images, columns=['fname'])

(70, 1)


,fname
0,(GalaxyN3)22.jpg
1,(GalaxyN3)23.jpg
2,(GalaxyN3)24.jpg
3,(GalaxyN3)25.jpg
4,(GalaxyS4)11.jpg


In [0]:
def feature_extract(m, lock):
    while True:
        images_list = m.get()
        if images_list is None:
            break
        if type(images_list) is tuple:
            img = imread('/home/maansi/input/check/train'/images_list[0]/images_list[1])
            key = images_list[1]
        else:
            img = imread('/home/maansi/input/check/'+images_list)
            key = images_list

        if img.shape == (2,):
            img = img[0]

        features[key] = (img[:, :, 0].mean(), img[:, :, 1].mean(), img[:, :, 2].mean(),
                           img[:, :, 0].std(),  img[:, :, 1].std(),  img[:, :, 2].std())    

In [0]:
feature_cols = ['channel1_mean', 'channel2_mean', 'channel3_mean', 'channel1_std', 'channel2_std', 'channel3_std']

for col in feature_cols:
    train[col] = None
    test[col] = None

In [0]:
parallel_cores = 8

features = proc.Manager().dict()

m = proc.Queue(maxsize=parallel_cores)
lock = proc.Lock()
process_pool = proc.Pool(parallel_cores, initializer=feature_extract, initargs=(m, lock))

for i in test_images:
    m.put(i)
    
for _ in range(parallel_cores):  
    m.put(None)
process_pool.close()
process_pool.join()

features = dict(features)

In [0]:
for n, col in enumerate(feature_cols):
    train[col] = train['fname'].apply(lambda x: features[x][n])
    test[col] = test['fname'].apply(lambda x: features[x][n])

In [0]:
y = train['camera'].values
X_train = train[cols].values
X_test = test[cols].values

In [0]:
clf = RandomForestClassifier(n_estimators=200)
clf.fit(X_train, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [0]:
filename = '/home/maansi/input/camera_model.pkl'
pickle.dump(clf, open(filename, 'wb'))